## Introdução

Aqui irei realizar a etapa de Feature Engineering na base de dados `instalments_payments`.

Esta etapa está dividida em arquivos diferentes devido o consumo de memória para carregar e manipular as bases.

Os procedimentos realizados nas outras bases podem ser encontrados nos arquivos `02_feat_eng_<nome-da-base>.ipynb`

O objetivo desta etapa consiste, principalmente, em criar variáveis (`book de variáveis`). Ao criar novas variáveis com base nas variáveis existentes, é possível capturar informações adicionais que podem não estar explicitamente presentes nos dados originais.

Devido o volume de dados, optei por utilizar o PySpark em conjunto com o SparkSQL para as operações a seguir.

* Sobre os Dados

A base `instalments_payments` possui informações sobre o histórico de pagamentos de empréstimos anteriores de um cliente.

Segundo os Metadados disponibilizados, essas são as informações contidas aqui:

`SK_ID_PREV`: ID do crédito anterior. (Um crédito em nossa amostra pode ter 0, 1, 2 ou mais créditos anteriores no Crédito Habitacional). Será utilizada como PK para conectarmos à base `previous_application`, conforme a esquematização abaixo.

`SK_ID_CURR`: ID do empréstimo.

`NUM_INSTALMENT_VERSION`: Versão do calendário de parcelamento (0 é para cartão de crédito) do crédito anterior. A mudança da versão do parcelamento de mês para mês significa que algum parâmetro do calendário de pagamento foi alterado.

`NUM_INSTALMENT_NUMBER`: Em qual parcela observamos o pagamento.

`DAYS_INSTALMENT`: Quando a parcela do crédito anterior deveria ser paga (em relação à data de solicitação do empréstimo atual).

`DAYS_ENTRY_PAYMENT`: Quando as parcelas do crédito anterior foram efetivamente pagas (em relação à data de solicitação do empréstimo atual).

`AMT_INSTALMENT`: Qual era o valor da parcela prescrito do crédito anterior nesta parcela.

`AMT_PAYMENT`: O que o cliente realmente pagou no crédito anterior nesta parcela.

## Utils

* Importando as bibliotecas que irei utilizar

In [1]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,min, max, sum, round, col, median, abs
spark = SparkSession.builder \
    .appName("FeatureEng") \
    .config("spark.executor.memory", "14g") \
    .config("spark.driver.memory", "14g") \
    .getOrCreate()
from warnings import filterwarnings
filterwarnings('ignore')

## Feature Engineering - Instalments Payments

### 1. Carregamento e Informações Gerais

* Carregando e Visualizando os dados utilizando o PySpark

In [2]:
instalments = spark.read.csv('./DATASETS/installments_payments.csv', header= True, inferSchema= True)

In [3]:
instalments.show(n=5, truncate= False)

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|1054186   |161674    |1.0                   |6                    |-1180.0        |-1187.0           |6948.36       |6948.36    |
|1330831   |151639    |0.0                   |34                   |-2156.0        |-2156.0           |1716.525      |1716.525   |
|2085231   |193053    |2.0                   |1                    |-63.0          |-63.0             |25425.0       |25425.0    |
|2452527   |199697    |1.0                   |3                    |-2418.0        |-2426.0           |24350.13      |24350.13   |
|2714724   |167756    |1.0                   |2                    |-1383.0        

* Verificando as dimensões (Linhas x Colunas)

In [4]:
(instalments.count(), len(instalments.columns))

(13605401, 8)

* Qtd. de Linhas após agrupamento (Para verificar ao final do processo)

In [5]:
instalments.groupBy('SK_ID_PREV').count().count()

997752

### 2. Criação de Flags Temporais

In [3]:
# Criando uma View
instalments.createOrReplaceTempView('instalments')

temp01 = spark.sql("""
SELECT
    *,
        CASE
            WHEN DAYS_INSTALMENT >= -90 THEN 1
        ELSE 0
    END AS FL_U3M,
        CASE
            WHEN DAYS_INSTALMENT >= -180 THEN 1
        ELSE 0
    END AS FL_U6M,
        CASE
            WHEN DAYS_INSTALMENT >= -270 THEN 1
        ELSE 0
    END AS FL_U9M,
        CASE
            WHEN DAYS_INSTALMENT >= -360 THEN 1
        ELSE 0
    END AS FL_U12M
FROM
    instalments
ORDER BY
    `SK_ID_PREV`;
""")

temp01.createOrReplaceTempView('temp01')

temp01.count()

13605401

### 3. Criando Variáveis Adicionais

Como `DAYS_INSTALMENT` se refere à data de pagamento esperada da parcela e `DAYS_ENTRY_PAYMENT` se refere à data que a parcela foi realmente paga, podemos criar algumas variáveis utilizando ambas as colunas.

* Criando a Var. `Dias de Atraso` (Qtd. de dias negativos indicam que a parcela foi paga antes da data esperada)

In [4]:
temp02 = temp01.withColumn("QTD_DAYS_ATRASO", abs(col('DAYS_ENTRY_PAYMENT')) - abs(col('DAYS_INSTALMENT')))
temp02.count()

13605401

* Criando a Var. `CAT_PGTO_PARCELA` ('ATRASO' se QTD_DIAS_ATRASO > 0, 'VENCIMENTO' se QTD_DIAS_ATRASO = 0, 'ANTECIPADO' se QTD_DIAS_ATRASO < 0)

In [8]:
temp02 = temp02.withColumn("FL_CAT_PGTO_ATRASADO", when(col('QTD_DAYS_ATRASO') > 0, 1).otherwise(0))
temp02 = temp02.withColumn("FL_CAT_PGTO_VENCIMENTO", when(col('QTD_DAYS_ATRASO') == 0, 1).otherwise(0))
temp02 = temp02.withColumn("FL_CAT_PGTO_ANTECIPADO", when(col('QTD_DAYS_ATRASO') < 0, 1).otherwise(0))
temp02.count()

13605401

### 4. Criação das Variáveis (Agrupadas)

* Filtrando as colunas que serão agregadas

In [5]:
agg_cols = [col for col in temp02.columns if ("FL_" not in col) & ("SK_ID" not in col)]

* Criando as novas variáveis e agrupando os dados

In [6]:
flags_temporais = ['FL_U3M', 'FL_U6M', 'FL_U9M','FL_U12M']

flags_categoricas = [col for col in temp02.columns if 'FL_CAT_' in col]

#### 4.1 Usando Apenas Flags Temporais

In [7]:
newcols = []

for flag_temp in flags_temporais:
    nome_flag_temp_tratado = flag_temp.replace("FL_",'')
    for agg_col in agg_cols:
        if 'DAY' in agg_col:
            newcols.append(round(max(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"QT_MAX_{agg_col}_{nome_flag_temp_tratado}_INSTALMENTS".upper()))
            newcols.append(round(min(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"QT_MIN_{agg_col}_{nome_flag_temp_tratado}_INSTALMENTS".upper()))
        else:
            newcols.append(round(max(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"VL_MAX_{agg_col}_{nome_flag_temp_tratado}_INSTALMENTS".upper()))
            newcols.append(round(min(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"VL_MIN_{agg_col}_{nome_flag_temp_tratado}_INSTALMENTS".upper()))
            newcols.append(round(median(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"VL_MED_{agg_col}_{nome_flag_temp_tratado}_INSTALMENTS".upper()))
            newcols.append(round(sum(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"VL_SUM_{agg_col}_{nome_flag_temp_tratado}_INSTALMENTS".upper()))

newcols = tuple(newcols)

temp03 = temp02.groupBy("SK_ID_PREV").agg(*newcols).orderBy("SK_ID_PREV")

temp03.count()

997752

In [8]:
print("Qtd. de Vars. Criadas:", len(temp03.columns))

Qtd. de Vars. Criadas: 89


In [13]:
temp03 = temp03.repartition(1)

temp03.write.mode("overwrite").option("compression", "gzip").parquet("./VARS/INSTALMENTS_PAYMENTS/FL_TEMPORAIS")

#### 4.2 Usando Apenas Flags Categóricas

In [14]:
newcols = []

for flag_categorica in flags_categoricas:
    nome_flag_cat_tratada = flag_categorica.replace("FL_CAT_",'')
    for agg_col in agg_cols:
        if 'DAY' in agg_col:
            newcols.append(round(max(when(col(flag_categorica) == 1, col(agg_col))),2).alias(f"QT_MAX_{agg_col}_{nome_flag_cat_tratada}_INSTALMENTS".upper()))
            newcols.append(round(min(when(col(flag_categorica) == 1, col(agg_col))),2).alias(f"QT_MIN_{agg_col}_{nome_flag_cat_tratada}_INSTALMENTS".upper()))
        else:
            newcols.append(round(max(when(col(flag_categorica) == 1, col(agg_col))),2).alias(f"VL_MAX_{agg_col}_{nome_flag_cat_tratada}_INSTALMENTS".upper()))
            newcols.append(round(min(when(col(flag_categorica) == 1, col(agg_col))),2).alias(f"VL_MIN_{agg_col}_{nome_flag_cat_tratada}_INSTALMENTS".upper()))
            newcols.append(round(median(when(col(flag_categorica) == 1, col(agg_col))),2).alias(f"VL_MED_{agg_col}_{nome_flag_cat_tratada}_INSTALMENTS".upper()))
            newcols.append(round(sum(when(col(flag_categorica) == 1, col(agg_col))),2).alias(f"VL_SUM_{agg_col}_{nome_flag_cat_tratada}_INSTALMENTS".upper()))

newcols = tuple(newcols)

temp04 = temp02.groupBy("SK_ID_PREV").agg(*newcols).orderBy("SK_ID_PREV")

temp04.count()

997752

In [15]:
print("Qtd. de Vars. Criadas:", len(temp04.columns))

Qtd. de Vars. Criadas: 67


In [16]:
temp04 = temp04.repartition(1)

temp04.write.mode("overwrite").option("compression", "gzip").parquet("./VARS/INSTALMENTS_PAYMENTS/FL_CATEGORICAS")

### 5. Juntando as Tabelas

In [17]:
# Verificando a quantidade de linha das tabelas

temp03.count(), temp04.count()

(997752, 997752)

In [18]:
# Verificando a quantidade de colunas das tabelas

len(temp03.columns), len(temp04.columns)

(89, 67)

* Após o Join, devemos esperar 997.752 linhas e 155 colunas

In [19]:
instalments_agg = temp03.join(other= temp04, on= 'SK_ID_PREV', how= 'left')

instalments_agg.count(), len(instalments_agg.columns)

(997752, 155)

* Salvando a tabela final

In [9]:
instalments_agg = temp03

instalments_agg = instalments_agg.repartition(1)

instalments_agg.write.mode('overwrite').parquet("./BASES_FEAT_ENG/INSTALMENTS_PAYMENTS_FEAT_ENG")

In [10]:
instalments_agg.count(), len(instalments_agg.columns)

(997752, 89)